In [3]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('data.csv')

In [13]:
print(df.columns)

Index(['asdf_id', 'udel_precip_v501_sum.1900.max',
       'udel_precip_v501_sum.1900.mean', 'udel_precip_v501_sum.1900.min',
       'udel_precip_v501_sum.1900.sum', 'udel_precip_v501_sum.1901.max',
       'udel_precip_v501_sum.1901.mean', 'udel_precip_v501_sum.1901.min',
       'udel_precip_v501_sum.1901.sum', 'udel_precip_v501_sum.1902.max',
       ...
       'Level', 'OBJECTID', 'Shape_Area', 'Shape_Length', 'gqid', 'id',
       'shapeGroup', 'shapeID', 'shapeName', 'shapeType'],
      dtype='object', length=14712)


In [17]:
print(df.describe())

          asdf_id  udel_precip_v501_sum.1900.max  \
count  735.000000                     731.000000   
mean   367.000000                    1674.061970   
std    212.320512                    1181.533295   
min      0.000000                     320.600000   
25%    183.500000                     963.400000   
50%    367.000000                    1394.000000   
75%    550.500000                    1940.100000   
max    734.000000                   11389.300000   

       udel_precip_v501_sum.1900.mean  udel_precip_v501_sum.1900.min  \
count                      731.000000                     731.000000   
mean                      1349.820865                    1077.470041   
std                        839.619154                     599.346487   
min                        219.167326                      71.800000   
25%                        791.701279                     652.850000   
50%                       1165.510224                     972.200000   
75%                       1

In [25]:
rainfall_cols = [col for col in df.columns if '.sum' in col]
rainfall_df = df[rainfall_cols]
print(rainfall_df)

     udel_precip_v501_sum.1900.sum  udel_precip_v501_sum.1901.sum  \
0                      1746.197027                    1589.226027   
1                      3856.902955                    3772.806962   
2                      5397.429014                    5695.707017   
3                      4540.424012                    3853.350010   
4                      3223.308988                    3215.012991   
..                             ...                            ...   
730                     421.520006                     756.120011   
731                    1011.860991                    1178.431988   
732                     466.539994                     434.495994   
733                    1021.776986                    1243.161985   
734                     901.607004                    1021.356004   

     udel_precip_v501_sum.1902.sum  udel_precip_v501_sum.1903.sum  \
0                      1669.170025                    1545.890024   
1                      3076.45796

In [27]:
df.isnull().sum()


asdf_id                           0
udel_precip_v501_sum.1900.max     4
udel_precip_v501_sum.1900.mean    4
udel_precip_v501_sum.1900.min     4
udel_precip_v501_sum.1900.sum     5
                                 ..
id                                0
shapeGroup                        0
shapeID                           0
shapeName                         0
shapeType                         0
Length: 14712, dtype: int64

In [36]:
# STEP 1: Select relevant columns
id_cols = ['district_id'] if 'district_id' in df.columns else ['asdf_id']
rainfall_sum_cols = [col for col in df.columns if 'udel_precip_v501_sum' in col and '.sum' in col]
rainfall_max_cols = [col for col in df.columns if 'udel_precip_v501_sum' in col and '.max' in col]
rainfall_min_cols = [col for col in df.columns if 'udel_precip_v501_sum' in col and '.min' in col]
ntl_cols = [col for col in df.columns if 'viirs_ntl_annual' in col and '.sum' in col]

# STEP 2: Create cleaned DataFrame
relevant_cols = id_cols + rainfall_sum_cols + rainfall_max_cols + rainfall_min_cols + ntl_cols
df_clean = df[relevant_cols].copy()

# STEP 3: Drop rows with missing values
df_clean.dropna(inplace=True)

# STEP 4: Reset index
df_clean.reset_index(drop=True, inplace=True)

# STEP 5: Check final result
print(df_clean.shape)

print(df_clean.describe())


(730, 364)
          asdf_id  udel_precip_v501_sum.1900.sum  \
count  730.000000                     730.000000   
mean   367.016438                    1763.212775   
std    212.865710                    1437.230045   
min      0.000000                       5.298000   
25%    182.250000                     853.108267   
50%    366.500000                    1373.105027   
75%    551.750000                    2263.898035   
max    734.000000                   12792.378870   

       udel_precip_v501_sum.1901.sum  udel_precip_v501_sum.1902.sum  \
count                     730.000000                     730.000000   
mean                     1634.813550                    1669.372231   
std                      1411.493814                    1374.401496   
min                         4.301000                       6.211000   
25%                       689.978759                     778.704248   
50%                      1228.125011                    1334.340505   
75%                    

In [38]:
df_clean.head()

,asdf_id,udel_precip_v501_sum.1900.sum,udel_precip_v501_sum.1901.sum,udel_precip_v501_sum.1902.sum,udel_precip_v501_sum.1903.sum,udel_precip_v501_sum.1904.sum,udel_precip_v501_sum.1905.sum,udel_precip_v501_sum.1906.sum,udel_precip_v501_sum.1907.sum,udel_precip_v501_sum.1908.sum,...,udel_precip_v501_sum.2017.min,viirs_ntl_annual_v20_cf_cvg.2018.sum,viirs_ntl_annual_v20_cf_cvg.2020.sum,viirs_ntl_annual_v20_cf_cvg.2017.sum,viirs_ntl_annual_v20_cf_cvg.2013.sum,viirs_ntl_annual_v20_cf_cvg.2019.sum,viirs_ntl_annual_v20_cf_cvg.2016.sum,viirs_ntl_annual_v20_cf_cvg.2012.sum,viirs_ntl_annual_v20_cf_cvg.2015.sum,viirs_ntl_annual_v20_cf_cvg.2014.sum
0,0,1746.197027,1589.226027,1669.170025,1545.890024,1898.988030,1032.992016,1825.752030,1272.178020,1677.868026,...,414.9,24287.910156,24287.910156,24287.910156,24287.910156,24287.910156,24287.910156,24287.910156,24287.910156,24287.910156
1,1,3856.902955,3772.806962,3076.457964,3767.658958,2731.828971,2933.621968,3264.712966,2941.286968,3695.390959,...,931.7,43130.368912,43130.368912,43130.368912,43130.368912,43130.368912,43130.368912,43130.368912,43130.368912,43130.368912
2,2,5397.429014,5695.707017,3603.031008,5725.852015,3748.222014,5610.012014,5974.209000,4058.621999,5322.234014,...,917.7,59509.051392,59509.051392,59509.051392,59509.051392,59509.051392,59509.051392,59509.051392,59509.051392,59509.051392
3,3,4540.424012,3853.350010,2954.436007,4172.623006,2446.859006,4104.907013,4385.198013,3223.457008,3609.610010,...,767.9,50411.950012,50411.950012,50411.950012,50411.950012,50411.950012,50411.950012,50411.950012,50411.950012,50411.950012
4,4,3223.308988,3215.012991,2244.900991,3040.748990,1997.566993,3082.792989,2775.988991,2446.914989,2607.395993,...,1085.4,33805.869141,33805.869141,33805.869141,33805.869141,33805.869141,33805.869141,33805.869141,33805.869141,33805.869141
